In [1]:
import numpy as np

In [2]:
E       = 150000            # Tegangan Sistem (volt)
h       = 3                 # depth / kedalaman penanaman konduktor (m)
p       = 100               # tanahan jenis tanah (ohm-m)
ps      = 3000              # tahanan permukaan tanah (koral) (ohm-m)
hs      = 0.1               # ketebalan lapisan koral
a       = 0.012             # diameter pembumian Mesh (m)
b       = 0.012             # diameter konduktor pembumian Rod (m)
aa      = np.sqrt(a*2*h)    # (a') untuk konduktor yang ditanam pada kedalaman h
d       = b
D       = 10                # jarak antar konduktor grid
A       = 6000              # luas area sistem pembumian (m^2)
K1      = 0.1
K2      = 4.5               
Lr      = 3                 # panjang batang konduktor Rod (m)
nR      = 32                # jumlah konduktor batang Rod
LR      = 100               # panjang total konduktor Rod (m)
LC      = 660               # panjang total konduktor Mesh (m)

Vf      = 120               # tegangan rms ke tanah (volt)
Z1      = 4                 # Impedansi positif (ohm)
TCAP    = 3.42              # faktor kapasitansi panas konduktor (J/cm3/*C)
ts      = 1                 # lama gangguan (detik)
ar      = 0.00393           # koefisien  panas resistivitas (*C)
pr      = 1.72              # resistivitas konduktor pentanahan (ohm/cm)
Ko      = 234               # Konstanta (1/ar) (*C)
Tm      = 1083              # suhu maksimum konduktor (*C)
Ta      = 34                # suhu tanah sekitar

Kii     = 1
m       = 6                 # jumlah_konduktor_sisi_panjang
n       = 10                # jumlah_konduktor_sisi_lebar

In [3]:
# RG (Resistansi Pentanahan) Grid-Rod (ohm)
R1 = (p/(np.pi*LC)) * (np.log(2*LC/aa) + (K1*LC/np.sqrt(A)) - K2)
R2 = (p/(2*np.pi*nR*LR)) * (np.log(4*LR/b) - 1 + (2*K1*Lr/np.sqrt(A)) * (np.sqrt(nR) - 1)**2)
Rm = (p/(np.pi*LC)) * (np.log(2*LC/Lr) + (K1*LC/np.sqrt(A)) - K2 + 1)
Rg = (R1*R2-Rm**2) / (R1+R2-2*Rm)

# Arus Gangguan Ke tanah (If (Amp))
If3 = Vf / Z1

# Luas penampang konduktor (mm2)
Amm2 = If3/(np.sqrt((TCAP*10**-4)/(ts*ar*pr)) * np.log((Ko+Tm)/(Ko+Ta)))


Cs = 1 - ((0.09*(1-(p/ps))) / 2*hs+0.09)
# Estep dan Etouch 50 Kg
Ik50 = 0.116 / np.sqrt(ts)
Es50 = (1000 + 6*Cs*ps) * (Ik50)
Et50 = (1000 + 1.5*Cs*ps) * (0.116/np.sqrt(ts))

# Estep dan Etouch 50 Kg
Ik70 = 0.157 / np.sqrt(ts)
Es70 = (1000 + 6*Cs*ps) * (Ik70)
Et70 = (1000 + 1.5*Cs*ps) * (0.157/np.sqrt(ts))

# Mesh Voltage (Em)
Kh = np.sqrt(1+(h/1))
N  = np.sqrt(n*m)
Ki = 0.644+0.148*N
Io = (3*(E/np.sqrt(3)))/(3*0+(4+4+10)+(10+10+40))
Ig = 0.6 * Io
IG = 1 * Ig
Km = (1 / (2 * np.pi)) * (np.log(D**2/(16*h*d)) + ((D+2*h)**2/(8*D*d)) - (h/(4*d)) + ((Kii/Kh) * np.log(8/(np.pi*(2*N-1)))))
LM = LC + LR
Em = ((p*Km*Ki*IG) / LM) / 100

# Step Voltage (Es)
Ks = (1/np.pi) * ((1/(2*h)) + (1/(D+h)) + ((1/D)*(1-(0.5**(N-2)))))
Es = (p*IG*Ks*Ki) / (0.75*LC + 0.85*LR)

#GPR
GPR = IG * Rg

In [5]:
print(Rg)

0.32710699499800316


In [6]:
LC      = 10*n + 10*m
Q       = m*n - ((m-2)*(n-2))
LR      = Q * 3
L       = LC + LR
Rg      = p * ((1/L) + ((1/np.sqrt(20*A)) * (1+(1/(1+h*np.sqrt(20/A))))))

Rg

0.9445680399570503

In [27]:
# # m       = 6
# # n       = 10
# # LC      = 10*n + 10*m
# # Q       = m*n - ((m-2)*(n-2))
# # LR      = Q * 3
# L       = LC + LR
LC = 0
LR = 0
Rg      = 0.3324

while Rg <= 10000:
    print(LC, end="")
    print(f"\t||  {LR}", end="")
    print(f"\t||  {Rg}")
    if 0.1 < Rg < 0.3:
        break
    # nR = np.random.randint(1, high=20)
    # h  = np.random.randint(1, high=3)

    LC += 1
    LR += 2

    # RG (Resistansi Pentanahan) Grid-Rod (ohm)
    aa = np.sqrt(a*2*h)
    R1 = (p/(np.pi*LC)) * (np.log(2*LC/aa) + (K1*LC/np.sqrt(A)) - K2)
    R2 = (p/(2*np.pi*nR*LR)) * (np.log(4*LR/b) - 1 + (2*K1*Lr/np.sqrt(A)) * (np.sqrt(nR) - 1)**2)
    Rm = (p/(np.pi*LC)) * (np.log(2*LC/Lr) + (K1*LC/np.sqrt(A)) - K2 + 1)
    Rg = (R1*R2-Rm**2) / (R1+R2-2*Rm)

    # Arus Gangguan Ke tanah (If (Amp))
    If3 = Vf / Z1

    # Luas penampang konduktor (mm2)
    Amm2 = If3/(np.sqrt((TCAP*10**-4)/(ts*ar*pr)) * np.log((Ko+Tm)/(Ko+Ta)))


    Cs = 1 - ((0.09*(1-(p/ps))) / 2*hs+0.09)
    # Estep dan Etouch 50 Kg
    Ik50 = 0.116 / np.sqrt(ts)
    Es50 = (1000 + 6*Cs*ps) * (Ik50)
    Et50 = (1000 + 1.5*Cs*ps) * (0.116/np.sqrt(ts))

    # Estep dan Etouch 50 Kg
    Ik70 = 0.157 / np.sqrt(ts)
    Es70 = (1000 + 6*Cs*ps) * (Ik70)
    Et70 = (1000 + 1.5*Cs*ps) * (0.157/np.sqrt(ts))

    # Mesh Voltage (Em)
    Kh = np.sqrt(1+(h/1))
    N  = np.sqrt(n*m)
    Ki = 0.644+0.148*N
    Io = (3*(E/np.sqrt(3)))/(3*0+(4+4+10)+(10+10+40))
    Ig = 0.6 * Io
    IG = 1 * Ig
    Km = (1 / (2 * np.pi)) * (np.log(D**2/(16*h*d)) + ((D+2*h)**2/(8*D*d)) - (h/(4*d)) + ((Kii/Kh) * np.log(8/(np.pi*(2*N-1)))))
    LM = LC + LR
    Em = ((p*Km*Ki*IG) / LM) / 100

    # Step Voltage (Es)
    Ks = (1/np.pi) * ((1/(2*h)) + (1/(D+h)) + ((1/D)*(1-(0.5**(N-2)))))
    Es = (p*IG*Ks*Ki) / (0.75*LC + 0.85*LR)

    #GPR
    GPR = IG * Rg


print("Final LC :", LC)
print("Final LR :", LR)
print("Final Rg :", Rg)

0	||  0	||  0.3324
1	||  2	||  -91.13016843364714
2	||  4	||  -35.38777940711687
3	||  6	||  -19.705158637529028
4	||  8	||  -12.747569053178216
5	||  10	||  -8.958529254312996
6	||  12	||  -6.634983203898337
7	||  14	||  -5.094760623083231
8	||  16	||  -4.015954204350149
9	||  18	||  -3.2286625771806756
10	||  20	||  -2.6355688473256165
11	||  22	||  -2.177332669715801
12	||  24	||  -1.8159348769277277
13	||  26	||  -1.52601847236364
14	||  28	||  -1.2900993509969025
15	||  30	||  -1.0957792132123527
16	||  32	||  -0.934053155430202
17	||  34	||  -0.7982441523365004
18	||  36	||  -0.6833110167045017
19	||  38	||  -0.5853865885460123
20	||  40	||  -0.5014621402432965
21	||  42	||  -0.42916711053339013
22	||  44	||  -0.36661245505386825
23	||  46	||  -0.31227734393684686
24	||  48	||  -0.26492595363923843
25	||  50	||  -0.22354550903404435
26	||  52	||  -0.1872995636534977
27	||  54	||  -0.15549236161172664
28	||  56	||  -0.12754136301254393
29	||  58	||  -0.10295585484257629
30	||  60	

In [28]:
print(f"Final nR                        : {nR} batang")
print(f"Arus Gangguan Ke Tanah (3Io)    : {If3} Amp")
print(f"Luas penampang konduktor        : {Amm2} mm2")

print("\n============Berat Badan 50 Kg============")
print(f"Arus Fibrasi                    : {Ik50} Amp")
print(f"Tegangan sentuh yang diizinkan  : {Es50} Volt")
print(f"Tegangan langkah yang diizinkan : {Et50} Volt")

print("\n============Berat Badan 70 Kg============")
print(f"Arus Fibrasi                    : {Ik70} Amp")
print(f"Tegangan sentuh yang diizinkan  : {Es70} Volt")
print(f"Tegangan langkah yang diizinkan : {Et70} Volt")

print("\n=============IEEE Grounding==============")
print(f"GPR                             : {GPR} volt")
print(f"Mesh Voltage (Em)               : {Em} volt")
print(f"Step Voltage (Es)               : {Es} volt")
print(f"Rg (resistansi pentanahan) rod  : {Rg} ohm")

Final nR                        : 32 batang
Arus Gangguan Ke Tanah (3Io)    : 30.0 Amp
Luas penampang konduktor        : 83.77059778260876 mm2

============Berat Badan 50 Kg============
Arus Fibrasi                    : 0.116 Amp
Tegangan sentuh yang diizinkan  : 2006.9972000000002 Volt
Tegangan langkah yang diizinkan : 588.7493000000001 Volt

============Berat Badan 70 Kg============
Arus Fibrasi                    : 0.157 Amp
Tegangan sentuh yang diizinkan  : 2716.3669 Volt
Tegangan langkah yang diizinkan : 796.841725 Volt

=============IEEE Grounding==============
GPR                             : 599.2183902342954 volt
Mesh Voltage (Em)               : 63.82294804258988 volt
Step Voltage (Es)               : 25.623015535003823 volt
Rg (resistansi pentanahan) rod  : 0.299831045717792 ohm
